In [2]:
import scanpy as sc
# adata_tagoe = sc.read(".h5ad")

In [25]:
sc.pp.normalize_total(adata_tagoe)
sc.pp.log1p(adata_tagoe)

In [10]:
sciplex=sc.read('.h5ad')

/home/MBDAI206AA201/anaconda3/envs/geneformer_ini/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
adata = sc.read('.h5ad')

In [6]:
with open('gene_list.txt', 'r') as f:
    gene_list = [line.strip() for line in f if line.strip()]

In [14]:
sciplex.var.index = sciplex.var.gene_short_name

In [15]:

missing_genes = [g for g in gene_list if g not in sciplex.var_names]
if missing_genes:
    raise ValueError(f"gene loss：{missing_genes}")

sciplex = sciplex[:, gene_list].copy()

/home/MBDAI206AA201/anaconda3/envs/geneformer_ini/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [9]:
sciplex.write("sci_plex_mini.h5ad")

In [3]:
gene_names = adata_tagoe.var.index.tolist()

with open('gene_list.txt', 'w') as f:
    for gene in gene_names:
        f.write(f"{gene}\n")

In [27]:
import os
os.sys.path.append("../")

In [28]:
from trainer._utils import train_valid_test
import torch
train_data, valid_data, test_data = train_valid_test(adata_tagoe, split_key = 'celltype_split1')

In [30]:
test_data.obs['cov_drug_dose'] = (
    test_data.obs['cell_type'].astype(str).str.cat(
        test_data.obs['drug'].astype(str), 
        sep='_'
    ).str.cat(
        test_data.obs['dose'].astype(str), 
        sep='_'
    )
)

/home/MBDAI206AA201/jupyter/yhz/sc/MOMDGDP-main/dataset/ipykernel_1572693/2294032573.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  test_data.obs['cov_drug_dose'] = (
/home/MBDAI206AA201/anaconda3/envs/geneformer_ini/lib/python3.9/site-packages/anndata/_core/anndata.py:1898: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/MBDAI206AA201/anaconda3/envs/geneformer_ini/lib/python3.9/site-packages/anndata/_core/anndata.py:1898: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [39]:
import os
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from scipy.sparse import issparse
from tqdm import tqdm

def yield_index_pairs(adata, batch=3000):
    if "paired_control_index" not in adata.obs:
        raise KeyError("missing paired_control_index")

    smiles_field = "SMILES" if "SMILES" in adata.obs else "drug_smiles"
    obs_to_idx = {name: i for i, name in enumerate(adata.obs_names)}

    target_mask = adata.obs["dose"] != 0
    target_idx_list = np.where(target_mask)[0]

    for i in range(0, len(target_idx_list), batch):
        batch_ids = target_idx_list[i:i+batch]
        result = []
        for tid in batch_ids:
            ctrl_name = str(adata.obs["paired_control_index"][tid])
            cid = obs_to_idx[ctrl_name]
            result.append((tid, cid))
        yield result, smiles_field
from pathlib import Path

def get_row(X, idx):
    row = X[idx]
    if issparse(row):
        return row.toarray().reshape(-1).astype("float32")
    else:
        return np.asarray(row).reshape(-1).astype("float32")


def write_parquet_shards(adata, save_dir, shard_size=3000):
    os.makedirs(save_dir, exist_ok=True)
    X = adata.X
    n_genes = adata.n_vars
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    schema = pa.schema({
        "target": pa.list_(pa.float32(), n_genes),
        "control": pa.list_(pa.float32(), n_genes),
        "smiles": pa.string(),
        "cov": pa.string()
    })

    shard_id = 0
    buffer = { "target": [], "control": [], "smiles": [], "cov": [] }

    for idx_batch, smiles_field in yield_index_pairs(adata):
        for tid, cid in tqdm(idx_batch, desc="Processing"):
            tgt = get_row(X, tid)
            ctrl = get_row(X, cid)

            buffer["target"].append(tgt)
            buffer["control"].append(ctrl)
            buffer["smiles"].append(str(adata.obs[smiles_field][tid]))
            buffer["cov"].append(str(adata.obs["cov_drug_dose"][tid]))

            if len(buffer["target"]) == shard_size:
                table = pa.Table.from_pydict(buffer, schema=schema)
                pq.write_table(table, os.path.join(save_dir, f"shard_{shard_id:05d}.parquet"))
                shard_id += 1
                buffer = { "target": [], "control": [], "smiles": [], "cov": [] }

    if len(buffer["target"]) > 0:
        table = pa.Table.from_pydict(buffer, schema=schema)
        pq.write_table(table, os.path.join(save_dir, f"shard_{shard_id:05d}.parquet"))

    print(f"✔ Saved all shards to {save_dir}")


In [40]:
write_parquet_shards( test_data, "test001")

Processing: 100%|██████████| 1740/1740 [00:00<00:00, 16556.50it/s]


✔ Saved all shards to /raid/MBDAI/tahoe/test001


In [41]:
test_data.var

""
gene_name
TSPAN6
TNMD
DPM1
SCYL3
FGR
...
LRTOMT
ARHGAP11B
FAM153B
